ИМПОРТ НУЖНЫХ БИБЛИОТЕК

In [56]:
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from docx import Document
import os
import numpy as np
import faiss
import chromadb
from chromadb import Client

РАЗБИВАЕМ БОЛЬШОЙ ТЕКСТОВЫЙ ДОКУМЕНТ НА ЧАНКИ ДЛИНОЙ 512

In [ ]:

# Загружаем токенизатор (используем BERT)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def chunk_text(text, max_tokens=512):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]


# Загрузка документов
documents = []
folder_path = "documents"
for filename in os.listdir(folder_path):
    if filename.endswith('.docx'):
        file_path = os.path.join(folder_path, filename)
        doc = Document(file_path)
        text = "\n".join([paragraph.text for paragraph in doc.paragraphs if paragraph.text.strip()])
        if text:
            documents.append(text)

# Разбиваем на чанки
chunks = []
for doc in documents:
    chunks.extend(chunk_text(doc))





Token indices sequence length is longer than the specified maximum sequence length for this model (4063 > 512). Running this sequence through the model will result in indexing errors


662

In [58]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
    

def chunks_to_emb(chunks):
    passages = []
    for chunk in chunks:
        passage = embedder.encode(chunk)
        passages.append(passage)
    return passages
    
passages = chunks_to_emb(chunks)
passages_arr = np.array(passages)
dimension = passages_arr.shape[1]
index = faiss.IndexFlatL2(dimension)

# Добавление эмбеддингов в индекс
for passage in passages_arr:
    # Reshape the passage to be a 2D array with shape (1, dimension)
    passage_reshaped = passage.reshape(1, -1)
    index.add(passage_reshaped)

In [68]:
# Поиск документов
def retrieve_documents(query, k=8):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(query_embedding, k)
    return [chunks[i] for i in indices[0]]

In [69]:
from transformers import T5ForConditionalGeneration, T5Tokenizer


# Загрузка генеративной модели (например, T5)
generator = T5ForConditionalGeneration.from_pretrained("t5-small")
generator_tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Функция для генерации ответа
def generate_answer(query, context):
    input_text = f"question: {query} context: {context}"
    input_ids = generator_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).input_ids
    outputs = generator.generate(input_ids, max_length=150, num_beams=5, early_stopping=True)
    return generator_tokenizer.decode(outputs[0], skip_special_tokens=True)


In [73]:
query = "Виды SLAM"

retrieved_docs = retrieve_documents(query)
context = " ".join(retrieved_docs)

answer = generate_answer(query,context)
print(f"Ответ:{answer} ")

Ответ:традиионнми алоритмами 


In [79]:
import os
from docx import Document

# Используем библиотеку для работы с файлами .docx
from docx import Document

# Импортируем необходимые библиотеки для обработки текста и создания эмбеддингов
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Загрузим модель для токенизации и эмбеддинга
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Функция для разбивки документа на части по количеству токенов
def chunk_text(text, max_tokens=512):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk) for chunk in chunks]

# Считываем документы из папки
documents = []
folder_path = "documents"
for filename in os.listdir(folder_path):
    if filename.endswith('.docx'):
        file_path = os.path.join(folder_path, filename)
        doc = Document(file_path)
        text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
        documents.append(text)

# Разбиваем каждый документ на фрагменты по 512 токенов
chunks = []
for document in documents:
    chunks.extend(chunk_text(document))

# Создаем эмбеддинги для каждого фрагмента
passages = []
for chunk in chunks:
    passage = embedder.encode(chunk)
    passages.append(passage)

# Преобразуем список эмбеддингов в массив NumPy
passages_arr = np.array(passages)
dimension = passages_arr.shape[1]

# Создадим индекс для поиска ближайших соседей
index = faiss.IndexFlatL2(dimension)

# Добавляем эмбеддинги в индекс
for passage in passages_arr:
    # Reshape the passage to be a 2D array with shape (1, dimension)
    passage_reshaped = passage.reshape(1, -1)
    index.add(passage_reshaped)

# Функция для поиска ближайших документов по запросу
def retrieve_documents(query, k=9):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(query_embedding, k)
    return [chunks[i] for i in indices[0]]

# Загрузка генеративной модели (например, T5)
from transformers import T5ForConditionalGeneration, T5Tokenizer

generator = T5ForConditionalGeneration.from_pretrained("t5-small")
generator_tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Функция для генерации ответа
def generate_answer(query, context):
    input_text = f"question: {query} context: {context}"
    input_ids = generator_tokenizer(input_text, return_tensors="pt").input_ids
    outputs = generator.generate(input_ids)
    return generator_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Пример использования
query = "Что такое SLAM?"

retrieved_docs = retrieve_documents(query)
context = " ".join(retrieved_docs)

answer = generate_answer(query, context)
print(f"Ответ: {answer}")

Token indices sequence length is longer than the specified maximum sequence length for this model (4063 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4207 > 512). Running this sequence through the model will result in indexing errors


Ответ: аи ар ааи


In [ ]:
from docx import Document
import os
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Чтение документа
document = Document('documents/article_b382e22c.docx')  # замените example.docx на ваш файл
text = '\n'.join(paragraph.text for paragraph in document.paragraphs)

# Разделение текста на предложения
sentences = text.split('\n')

# Создание эмбеддингов для предложений
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

# Сохраняем эмбеддинги и соответствующие им предложения
data = list(zip(embeddings, sentences))

# Создание индекса
d = embeddings[0].shape[0]  # размерность эмбеддингов
index = faiss.IndexFlatIP(d)  # используем косинусную меру близости
index.add(np.array([emb for emb, _ in data]))

def search_relevant_sentences(query, k=5):
    query_embedding = model.encode([query])[0]
    _, idx = index.search(np.array([query_embedding]), k)
    relevant_sentences = [data[i][1] for i in idx[0]]
    return relevant_sentences

# Загрузка генератора
generator_model = T5ForConditionalGeneration.from_pretrained("t5-small")
generator_tokenizer = T5Tokenizer.from_pretrained("t5-small")

def generate_response(question, context):
    input_text = f"question: {question} context: {context}"
    inputs = generator_tokenizer(input_text, return_tensors='pt', padding=True)
    output = generator_model.generate(**inputs)
    response = generator_tokenizer.decode(output[0], skip_special_tokens=True)
    return response

if __name__ == "__main__":
    while True:
        question = input("Введите ваш вопрос: ")
        
        # Поиск релевантных предложений
        relevant_context = search_relevant_sentences(question)
        
        # Формирование контекста
        context = ' '.join(relevant_context)
        
        # Генерация ответа
        answer = generate_response(question, context)
        
        print(f"Ответ: {answer}")

Ответ: ододе среда раиваетс на с
Ответ: ододе среда раиваетс
Ответ: еросетва реалиаи
Ответ: wf
Ответ: Wefweg
Ответ: wegwegwevwevewv
Ответ: ис.1 лоритм Feature - based SL
Ответ: ис.1 лоритм Feature - based SL
Ответ: ис.1 лоритм Feature - based SL
Ответ: ис.1 лоритм Feature - based SL
Ответ: ис.2
Ответ: ис.1
Ответ: 
Ответ: 
Ответ: 
